# Fluffy recognition model

Trying to build a model that recognises whether an image has something fluffy in it.

Install the fastai library

In [1]:
!pip install fastai==2.5

     |████████████████████████████████| 188 kB 5.0 MB/s 
     |████████████████████████████████| 56 kB 5.1 MB/s 
  Attempting uninstall: fastai
    Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


Download images

In [2]:
from fastai.vision.all import untar_data, ImageDataLoaders, get_image_files, Resize, cnn_learner, resnet18, error_rate

IMG_URL = "https://github.com/mihailthebuilder/fluffy-nb/raw/main/fluffy-images.tar.xz"
path = untar_data(IMG_URL)

Check if download successful

In [3]:
get_image_files(path)

(#283) [Path('/root/.fastai/data/fluffy-images/NATURAL-GAS-VS-PROPANE.jpg'),Path('/root/.fastai/data/fluffy-images/fvfrfn6.jpg'),Path('/root/.fastai/data/fluffy-images/fluffy-cat-funny-pics-31__605.jpg'),Path('/root/.fastai/data/fluffy-images/GETTYIMAGES-976208412.jpg'),Path('/root/.fastai/data/fluffy-images/35-SEWING-NEEDLE.JPEG.jpeg.jpg'),Path('/root/.fastai/data/fluffy-images/wvjoxyi.jpg'),Path('/root/.fastai/data/fluffy-images/DOWNLOAD (8).jpg'),Path('/root/.fastai/data/fluffy-images/BOTTLES_7.jpg'),Path('/root/.fastai/data/fluffy-images/MERLIN_195320133_667F4C5E-8088-404C-ACDE-A1102C55FDB3-MOBILEMASTERAT3X.jpg'),Path('/root/.fastai/data/fluffy-images/OPERATION_UPSHOT-KNOTHOLE_-_BADGER_001.jpg')...]

Prepare data for model training

In [4]:
def is_fluffy(x): return x[0].isupper()
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_fluffy, item_tfms=Resize(224))

For training our model, we start by fetching the pre-trained ResNet model architecture with 18 layers. We then fine tune this model to our task in 2 epochs.

In [5]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(2)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch,train_loss,valid_loss,error_rate,time
0,1.276821,1.320788,0.232143,00:04


epoch,train_loss,valid_loss,error_rate,time
0,1.025450,0.738912,0.267857,00:04
1,0.856888,0.569476,0.250000,00:04


The error rate for the model should be somewhere around 20-25%. This was the best accurace I could achieve after...
- Increasing the number of images fed into it (currently 283).
- Tweaking the number of epochs.
- Changing the number of layers in the ResNet model architecture I was using.